# Introduction

This notebook explores the shapelet transformers implemented within aeon. For a wider understanding please refer to antoines notebook first.

if you want to learn about x y z then go to antoines

here youll learn about a b c

# The three transformers

In [20]:
from aeon.registry import all_estimators

for k, v in all_estimators("transformer", filter_tags={"algorithm_type": "shapelet"}):
    print(f"{k}: {v}")

RSAST: <class 'aeon.transformations.collection.shapelet_based._rsast.RSAST'>
RandomDilatedShapeletTransform: <class 'aeon.transformations.collection.shapelet_based._dilated_shapelet_transform.RandomDilatedShapeletTransform'>
RandomShapeletTransform: <class 'aeon.transformations.collection.shapelet_based._shapelet_transform.RandomShapeletTransform'>


# A little bit about each transformer
- talk generally about the purpose of transforms as opposed to shapelet trees
- talk about their params and logic and contributions and flaws (write in order of release)

## Shapelet transform

## Random Dilated Shapelet Transform

## Scalable and Accurate Subsequence Transform

# The datasets to be explored

### (Univariate) Gun/No Gun

Write some stuff from the very first shapelet paper and any other paper references to gun no gun

In [21]:
from aeon.datasets import load_classification

X_gun_train, y_gun_train = load_classification("GunPoint", split="train")
X_gun_test, y_gun_test = load_classification("GunPoint", split="test")

print(f"shape of the array: {X_gun_train.shape}")
print(f"n_samples = {X_gun_train.shape[0]}")
print(f"n_channels = {X_gun_train.shape[1]}")
print(f"n_timepoints = {X_gun_train.shape[2]}")

shape of the array: (50, 1, 150)
n_samples = 50
n_channels = 1
n_timepoints = 150


### (Multivariate) Motion

This human activity recognition dataset is time series data generated by accelerometer and gyroscope sensors (attitude, gravity, user acceleration, and rotation rate) used in the paper {Malekzadeh et al., Mobile Sensor Data Anonymization, in proc the International Conference on Internet of Things Design and Implementation, 2019 (Link Here) A total of 24 participants in a range of gender, age, weight, and height performed 6 activities in 15 trials in the same environment and conditions: downstairs (dws), upstairs , walking, jogging, sitting, and standing.

In [22]:
from aeon.datasets import load_classification

X_motion_train, y_motion_train = load_classification("MotionSenseHAR", split="train")
X_motion_test, y_motion_test = load_classification("MotionSenseHAR", split="test")

print(f"shape of the array: {X_motion_train.shape}")
print(f"n_samples = {X_motion_train.shape[0]}")
print(f"n_channels = {X_motion_train.shape[1]}")
print(f"n_timepoints = {X_motion_train.shape[2]}")

shape of the array: (966, 12, 1000)
n_samples = 966
n_channels = 12
n_timepoints = 1000


# Discuss how we will explore the transform, compare them, and hopefully understand something about the motion data

- track how long it takes to fit the transform
- compare the shapelets
    - compare their discriminative ability

- run through different transformers with different configurations
- plot the output so the shapelet generated and the transformed values
- ST take the basic motion data, find the shapelets and then plot back where the shapelet is in the motion
- useful visualisation tools are ShapeletVisualizer, ShapeletTransformVisualizer

# Fitting the training data

## Fitting each transform to training data with default parameters

### (Univariate) ___ dataset

#### Random Shapelet Transform

In [23]:
import time
from aeon.transformations.collection.shapelet_based import RandomShapeletTransform

# Start the timer
start_time = time.time()

rst_univar = RandomShapeletTransform(max_shapelets=10).fit(X_gun_train, y_gun_train)
# Stop the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to fit: {elapsed_time:.4f} seconds")

rst_univar.transform(X_gun_test).shape

Time taken to fit: 5.7962 seconds


(150, 10)

#### Random Dilated Shapelet Transform

In [24]:
from aeon.transformations.collection.shapelet_based import RandomDilatedShapeletTransform
import time
# Start the timer
start_time = time.time()

rsdt_univar = RandomDilatedShapeletTransform(max_shapelets=10000).fit(X_gun_train, y_gun_train)
# Stop the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to fit: {elapsed_time:.4f} seconds")

rsdt_univar.transform(X_gun_test).shape

Time taken to fit: 0.7918 seconds


(150, 30000)

#### Scalable and Accurate Subsequence Transform

In [25]:
from aeon.transformations.collection.shapelet_based import SAST
import time
# Start the timer
start_time = time.time()

sast = SAST().fit(X_gun_train, y_gun_train)
# Stop the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to fit: {elapsed_time:.4f} seconds")

sast.transform(X_gun_test).shape

Time taken to fit: 0.1564 seconds


(150, 22050)

### (Multivariate) Motion dataset

#### Random Shapelet Transform

In [ ]:
import time
from aeon.transformations.collection.shapelet_based import RandomShapeletTransform

# Start the timer
start_time = time.time()

rst_multivar = RandomShapeletTransform(max_shapelets=10).fit(X_motion_train, y_motion_train)
# Stop the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to fit: {elapsed_time:.4f} seconds")

rst_multivar.transform(X_motion_test).shape

#### Random Dilated Shapelet Transform

In [ ]:
from aeon.transformations.collection.shapelet_based import RandomDilatedShapeletTransform
import time
# Start the timer
start_time = time.time()

rsdt_multivar = RandomDilatedShapeletTransform(max_shapelets=10000).fit(X_motion_train, y_motion_train)
# Stop the timer
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to fit: {elapsed_time:.4f} seconds")

rsdt_multivar.transform(X_motion_test).shape

## Fitting each visualizer

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
st_vis.plot(
    id_shapelet,
    ax=ax[0],
    scatter_options={"c": "purple"},
    line_options={"linestyle": "-."},
)
st_vis.plot_on_X(
    id_shapelet, X_test[1], ax=ax[1], line_options={"linewidth": 3, "alpha": 0.5}
)
ax[1].set_title(f"Best match of shapelet {id_shapelet} on X")
st_vis.plot_distance_vector(
    id_shapelet, X_test[1], ax=ax[2], line_options={"c": "brown"}
)
ax[2].set_title(f"Distance vector of shapelet {id_shapelet} on X")

### (Multivariate) motion dataset

In [ ]:
from aeon.visualisation import ShapeletTransformerVisualizer

st_vis = ShapeletTransformerVisualizer(st)
id_shapelet = 0  # Identifier of the shapelet

fig = st_vis.plot_on_X(id_shapelet, X_test[1], figure_options={"figsize": (7, 4)})

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
st_vis.plot(
    id_shapelet,
    ax=ax[0],
    scatter_options={"c": "purple"},
    line_options={"linestyle": "-."},
)
st_vis.plot_on_X(
    id_shapelet, X_test[1], ax=ax[1], line_options={"linewidth": 3, "alpha": 0.5}
)
ax[1].set_title(f"Best match of shapelet {id_shapelet} on X")
st_vis.plot_distance_vector(
    id_shapelet, X_test[1], ax=ax[2], line_options={"c": "brown"}
)
ax[2].set_title(f"Distance vector of shapelet {id_shapelet} on X")

### (Univariate) ___ dataset

In [ ]:
from aeon.visualisation import ShapeletTransformerVisualizer

st_vis = ShapeletTransformerVisualizer(st)
id_shapelet = 0  # Identifier of the shapelet

fig = st_vis.plot_on_X(id_shapelet, X_test[1], figure_options={"figsize": (7, 4)})

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
st_vis.plot(
    id_shapelet,
    ax=ax[0],
    scatter_options={"c": "purple"},
    line_options={"linestyle": "-."},
)
st_vis.plot_on_X(
    id_shapelet, X_test[1], ax=ax[1], line_options={"linewidth": 3, "alpha": 0.5}
)
ax[1].set_title(f"Best match of shapelet {id_shapelet} on X")
st_vis.plot_distance_vector(
    id_shapelet, X_test[1], ax=ax[2], line_options={"c": "brown"}
)
ax[2].set_title(f"Distance vector of shapelet {id_shapelet} on X")

# Interpreting the data